## 4.4.a Building your own feed-forward network

Import numpy, which is really all we need to create our own NN.

In [1]:
import numpy as np

Recall that our simple neural network consisted of two layers. We also added a `ReLU` function as a non-linearity to the output of our intermediate layer. Given an input $\mathbf{x} \in \mathbb{R}^n $ we have

$ \mathbf{h} = f^{(1)}(\mathbf{x}; \mathbf{W},c) = ReLU(\mathbf{W}^\mathsf{T} \mathbf{x} + c) $ 

$ \mathbf{y} = f^{(2)}(\mathbf{h}; \mathbf{w},b) = \text{$ softmax $}( \mathbf{w}^\mathsf{T} \mathbf{h} + b) $

In this exercise you will create your own network. However, we will do it in a way that allows you to specify the depth of network, i.e. we extend our network such that there isn't just one $\mathbf{h}$ intermediate layers, but rather $n$ of them $\mathbf{h}_{i}$ with $i \in \{1,..., n\}$

**NOTE**: You are not allowed to use any built-in functions to calculate the ReLU, Softmax or the forward pass directly.

**NOTE 2**: Remember to include the non-linearity at every layer. Remember to also add the bias to every layer. Finally, remember to apply the softmax in the output layer.

In [2]:
def relu(x):
    """
    Implement the ReLU function as defined in the lecture
    Input: an array of numbers
    Output: ReLU(x)
    """
    for i in range(len(x)):
        x[i] = max(0, x[i])
    return x

In [3]:
def softmax(x):
    """
    Implement the `softmax` function as defined in the lecture
    """
    return np.exp(x)/sum(np.exp(x))

In [4]:
class FFNetwork:
    """
    Class representing the feed-forward neural network
    """
    def __init__(self, input_dim: int, hidden_dim: int,
                 output_dim: int, hidden_size: int):
        """
        Args:
        input_dim: dimensionality of `x`
        hidden_dim: dimensionality of the intermediate `h_i`
        output_dim: dimensionality of `y`
        hidden_size: number of intermediate layers `h_i`
        """
        # TODO: Implement
        # Initialize each layer as a random matrix of the
        # appropriate dimensions
        
        ## SOLUTION ##
        
        self.hidden_size = hidden_size
        
        setattr(self, f"layer_input", np.random.rand(hidden_dim, input_dim))
        setattr(self, f"bias_input", np.random.rand(hidden_dim))
        
        for i in range(hidden_size):
            setattr(self, f"layer_{i}", np.random.rand(hidden_dim, hidden_dim))
            setattr(self, f"bias_{i}", np.random.rand(hidden_dim))
        
        setattr(self, "layer_output", np.random.rand(output_dim, hidden_dim))
        setattr(self, f"bias_output", np.random.rand(output_dim))
        
        ## SOLUTION ##
    
    def forward(self, x):
        """
        Args:
        x: input to the neural network
        
        Output:
        `y`, i.e. the prediction of the network
        
        Note: Remember to apply the ReLU and add the bias for each layer
        """
        # TODO: Implement the forward pass of the network,
        # i.e. calculate `y` from an input `x`
        # Remember that each layer's output is calculated by
        # f^(i) = ReLU(W_i^T * f^(i-1) + b_i)
        res = x
        
        ## SOLUTION ##
        
        res = relu(np.dot(getattr(self, "layer_input"), res) + getattr(self, "bias_input"))
        
        for i in range(self.hidden_size):
            res = relu(np.dot(getattr(self, f"layer_{i}"), res) + getattr(self, f"bias_{i}"))
        
        res = softmax(np.dot(getattr(self, "layer_output"), res) + getattr(self, "bias_output"))
        
        ## SOLUTION ##
        
        return res

Your implementation needs to be compatible with the following test code:

In [5]:
np.random.seed(0)

# A configuration that reflects the example from the lecture
# i.e. our input is of size 2, our intermediate layers are also of size 2,
# and we will only have 1 hidden layer.
network = FFNetwork(2, 2, 2, 1)
network.forward([1.,0.])

array([0.87832686, 0.12167314])

Disclaimer: Do not expect a correct output at this stage, you are simply building the structure of the network.

However, our setup also allows us to create larger networks:

In [6]:
np.random.seed(0)

network = FFNetwork(2, 3, 2, 4)
network.forward([1.,0.]) 

array([0.15216092, 0.84783908])

Some sanity checks:

1. You should be seeing the number of units you specified as output units in your output.
1. The numbers in your outputs should be in the range $[0,1]$
1. The numbers should add up to $1$
1. Varying the structure of the network should not break its functionality.

## 4.4.b Implementing a feed-forward network using `torch`

### 4.4.b.1 Creating the network (1 point)

For this we will be using the `nn` module of `torch`, which contains modules representing types of layers. In your case, the specific relevant module would be that of a *fully connected linear layer*.

We will also be using the `nn.functional` module to take advantage of the built in functions for ReLU and Softmax. In this exercise, you are allowed to use them.

In [7]:
import torch
import torch.nn.functional as F

from torch import nn

In [22]:
class TorchFFNetwork(nn.Module):
    """
    A `torch` version of the network implemented for 4.3.b
    """
    def __init__(self, input_dim: int, hidden_dim: int,
                 output_dim: int, hidden_size: int):
        """
        Args:
        input_dim: dimensionality of `x`
        hidden_dim: dimensionality of the intermediate `h_i`
        output_dim: dimensionality of `y`
        hidden_size: number of intermediate layers `h_i`
        """
        ## SOLUTION ##
        super(TorchFFNetwork, self).__init__()
        
        self.hidden_size = hidden_size
        
        setattr(self, f"layer_input", nn.Linear(input_dim, hidden_dim, bias=True))
        
        for i in range(hidden_size):
            setattr(self, f"layer_{i}", nn.Linear(hidden_dim, hidden_dim, bias=True))
        
        setattr(self, "layer_output", nn.Linear(hidden_dim, output_dim, bias=True))

        ## SOLUTION ##

    def forward(self, x):
        ## SOLUTION ##
        res = x

        res = getattr(self, "layer_input")(res)
        res = F.relu(res)

        for i in range(self.hidden_size):
            res = getattr(self, f"layer_{i}")(res)
            res = F.relu(res)

        res = getattr(self, "layer_output")(res)
        res = F.softmax(res)
        
        return res
        ## SOLUTION ##
 


Your implementation, once more, needs to be compatible with the following test code:

In [23]:
torch_network = TorchFFNetwork(2, 2, 2, 1)

In [24]:
with torch.no_grad():
    print(torch_network(torch.tensor([1.,0.])))

tensor([0.6288, 0.3712])


/home/carbon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Note that the `forward` method is automatically called when you call your network object.

### 4.4.b.2 Training your network (1 point)

Even though we have not covered how training actually works, we will proceed with the training of the a neural network as a blackbox procedure and we will later on learn the internals of the training process (and even implement them ourselves!).

For now, train a neural network (the one you created above) to learn the XOR operation. You are to create a neural network with the appropriate number of input variables, an intermediate hidden layer with 2 units and an output layer with 2 units.

Notes:
- Please read [this introduction to the optimization loop in PyTorch](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#optimization-loop). It should give you a good overview to what PyTorch needs from you to train a neural network.
- You are to train the network until the network learns the operation. Remember to set your random seeds so the results are reproducible.
- There are many optimizers available and Adam is an optimizer that's more complex than SGD. It has not yet been covered in the lecture but its usage in code is equivalent to that of SGD and performs much better.

In [25]:
# Our training X, where each instance includes an x1 and an x2, (where the operation is defined as x1 XOR x2)
training_x = [[0,0], [0,1], [1,0], [1,1]]

# We have only covered softmax in the lecture, so we format the output as follows:
training_y = [[1,0], [0,1], [0,1], [1,0]]

# The Y is formatted such that the its first element corresponds to the probability of the XOR resulting in a 0
# and the second element to the probability of the XOR resulting in a 1

################################################################
# TODO: Adapt the training set so it can be used with `pytorch`
################################################################
from torch.utils.data import DataLoader , TensorDataset

training_x = torch.tensor(training_x, dtype=torch.float, requires_grad=True)
training_y = torch.tensor(training_y, dtype=torch.float, requires_grad=True)

dataset = TensorDataset(training_x, training_y)

data_loader = DataLoader(dataset, batch_size=4)

In [26]:
# Create the model from the previous class and pick a learning rate
torch.manual_seed(42)
model = TorchFFNetwork(2, 2, 2, 2)
learning_rate = 0.0001

In [31]:
def train_loop(data, model, loss_fn, optimizer):
    # TODO: Implement
    for batch, (xs, ys) in enumerate(data):
        pred = model.forward(xs)
        loss = loss_fn(pred, ys)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), batch * len(xs)
        print(f"loss: {loss:>7f}")

In [32]:
# TODO: Run training
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params': model.parameters()}
], lr=learning_rate)


epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(data_loader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 0.706652
Epoch 2
-------------------------------
loss: 0.706633
Epoch 3
-------------------------------
loss: 0.706613
Epoch 4
-------------------------------
loss: 0.706594
Epoch 5
-------------------------------
loss: 0.706575
Epoch 6
-------------------------------
loss: 0.706556
Epoch 7
-------------------------------
loss: 0.706536
Epoch 8
-------------------------------
loss: 0.706517
Epoch 9
-------------------------------
loss: 0.706498
Epoch 10
-------------------------------
loss: 0.706478
Done!


/home/carbon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
